# Arquivo JSON

## Inicialmente, abri o arquivo para perceber quais tipos de informações ele continha. Percebi que ele estava humanamente ilegivel e precisava de um tratamento para ficar com o formato legível de que arquivos JSON comumente apresentam. Através de uma ferramenta simples do Python, fiz a conversão do arquivo, possibilitando ler o arquivo.

### cat receitas.json | python -m json.tool > receitas_pretty.json

## Assim, foi possível perceber que o arquivo se tratava de um dicionário de receitas, contendo informações como o passo-a-passo para fazer o prato, ingredientes, informações de tabela nutricional, nome da receita e a nota dada pelos usuários do site (www.epicurious.com)

## A partir de observações nos primeiros exemplos, pude perceber que a estrutura de cada elemento, continha as seguintes informações:

1. Directions: uma [lista] de passos a serem seguidos para elaborar o prato;
2. Fat: um float com a quantidade de gordura do prato;
3. Date: uma string contendo a data em que a receita foi divulgada (aparentemente, mas é possível que seja outra coisa);
4. Categories: [lista] de categorias às quais o prato pertence;
5. Calories: float com o número de calorias;
6. Desc: string com uma breve descrição do prato ou uma informação relacionada a ele;
7. Protein: float contendo a quantidade de proteínas presentes;
8. Rating: float contendo a nota do prato (imagino que dada pelos usuários do site);
9. Title: string contendo o nome do prato;
10. Ingredients: [lista] contendo os ingredientes utilizados na receita;
11. Sodium: float com a quantidade de sal presente no prato.

## Depois de entender as informações fornecidas, decidi prosseguir realizando um parse das informações, gerando um Numpy array contendo somente as informações úteis para os objetivos finais do desafio.

In [1]:
import numpy as np
import pandas as pd
import json
from collections import OrderedDict
# from IPython.display import display, HTML

receitas_path = 'receitas.json'

def parse(file_path):
    with open(file_path, 'r') as f:
        opt = json.load(f, object_pairs_hook=OrderedDict)
        
    return opt

opt = parse(receitas_path)

data = np.asarray(opt, dtype='object')
receitas_np = np.empty((len(data), 8), dtype='object')
wanted_keys = ['title', 'categories', 'calories', 'ingredients', 'fat', 'protein', 'sodium', 'rating']
for i in range(len(data)):
    for key in data[i].keys():
        if key in wanted_keys:
            if key == 'title':
                receitas_np[i, 0] = data[i][key]
            elif key == 'categories':
                receitas_np[i, 1] = data[i][key]
            elif key == 'calories':
                receitas_np[i, 2] = data[i][key]
            elif key == 'ingredients':
                receitas_np[i, 3] = data[i][key] 
            elif key == 'fat':
                receitas_np[i, 4] = data[i][key]            
            elif key == 'protein':
                receitas_np[i, 5] = data[i][key]
            elif key == 'sodium':
                receitas_np[i, 6] = data[i][key]
            elif key == 'rating':
                receitas_np[i, 7] = data[i][key]
rows_none, _ = np.where(receitas_np == None)
receitas_np = np.delete(receitas_np, rows_none, 0)

In [2]:
df = pd.DataFrame(receitas_np).drop_duplicates(subset=0)
display(df.rename(columns={
    0: 'title',
    1: 'categories',
    2: 'calories',
    3: 'ingredients',
    4: 'fat',
    5: 'protein',
    6: 'sodium',
    7: 'rating'
}).sort_values('calories'))

,title,categories,calories,ingredients,fat,protein,sodium,rating
7367,Spritz Wreaths,"[Cookies, Dairy, Dessert, Bake, Winter, Gourmet]",0,[1/2 preparedBasic Butter Cookie Dough at room...,0,0,1,2.5
3883,To Clarify Butter,"[Dairy, Quick & Easy, Gourmet]",0,[Unsalted butter],0,0,0,5
1324,Sriracha Salt,"[Condiment, Hot Pepper, Chile Pepper, No-Cook]",0,"[1 to 2 tablespoons sriracha sauce, 1 cup unre...",0,0,2391,0
4860,Dashi (Japanese Sea Stock),"[Soup/Stew, Spring, Gourmet]",0,"[6 cups cold water, 1 oz (30 grams) kombu (dri...",0,0,26,5
12013,Fresh Vegetable Platter with Olive Oil Dip,"[Vegetable, Appetizer, No-Cook, Prosciutto, Bo...",1,"[Assorted cut-up vegetables (such as carrots, ...",0,0,42,3.75
...,...,...,...,...,...,...,...,...
1037,"Rice Pilaf with Lamb, Carrots, and Raisins","[Lamb, Rice, Vegetable, Braise, Dinner, Raisin...",4.15736e+06,"[3 cups basmati rice (7 ounces), 1 medium onio...",221495,236489,3.13485e+06,5
2362,Lamb Köfte with Tarator Sauce,"[Food Processor, Backyard BBQ, Ground Lamb, Sp...",4.51822e+06,"[3 slices firm white sandwich bread, torn into...",44198,166471,7.54099e+06,5
15585,"Apricot, Cranberry and Walnut Pie","[Berry, Fruit, Nut, Dessert, Bake, Thanksgivin...",1.30629e+07,"[1 cup (packed) golden brown sugar, 2 large eg...",747374,87188,1.20058e+07,4.375
5086,Deep-Dish Wild Blueberry Pie,"[Egg, Fruit, Bake, Blueberry, Summer, Tapioca,...",2.99979e+07,"[1 1/4 cups packed light brown sugar, 5 tables...",1.71628e+06,200210,2.7571e+07,4.375


In [3]:
df.rename(columns={
    0: 'title',
    1: 'categories',
    2: 'calories',
    3: 'ingredients',
    4: 'fat',
    5: 'protein',
    6: 'sodium',
    7: 'rating'
}).sort_values('calories')

,title,categories,calories,ingredients,fat,protein,sodium,rating
7367,Spritz Wreaths,"[Cookies, Dairy, Dessert, Bake, Winter, Gourmet]",0,[1/2 preparedBasic Butter Cookie Dough at room...,0,0,1,2.5
3883,To Clarify Butter,"[Dairy, Quick & Easy, Gourmet]",0,[Unsalted butter],0,0,0,5
1324,Sriracha Salt,"[Condiment, Hot Pepper, Chile Pepper, No-Cook]",0,"[1 to 2 tablespoons sriracha sauce, 1 cup unre...",0,0,2391,0
4860,Dashi (Japanese Sea Stock),"[Soup/Stew, Spring, Gourmet]",0,"[6 cups cold water, 1 oz (30 grams) kombu (dri...",0,0,26,5
12013,Fresh Vegetable Platter with Olive Oil Dip,"[Vegetable, Appetizer, No-Cook, Prosciutto, Bo...",1,"[Assorted cut-up vegetables (such as carrots, ...",0,0,42,3.75
...,...,...,...,...,...,...,...,...
1037,"Rice Pilaf with Lamb, Carrots, and Raisins","[Lamb, Rice, Vegetable, Braise, Dinner, Raisin...",4.15736e+06,"[3 cups basmati rice (7 ounces), 1 medium onio...",221495,236489,3.13485e+06,5
2362,Lamb Köfte with Tarator Sauce,"[Food Processor, Backyard BBQ, Ground Lamb, Sp...",4.51822e+06,"[3 slices firm white sandwich bread, torn into...",44198,166471,7.54099e+06,5
15585,"Apricot, Cranberry and Walnut Pie","[Berry, Fruit, Nut, Dessert, Bake, Thanksgivin...",1.30629e+07,"[1 cup (packed) golden brown sugar, 2 large eg...",747374,87188,1.20058e+07,4.375
5086,Deep-Dish Wild Blueberry Pie,"[Egg, Fruit, Bake, Blueberry, Summer, Tapioca,...",2.99979e+07,"[1 1/4 cups packed light brown sugar, 5 tables...",1.71628e+06,200210,2.7571e+07,4.375


## Assim, receitas_np contém todos os elementos que considerei importantes para a resolução de alguma questão do desafio. Ele também contém missing values, que serão tratados em cada aplicação. Para a solução dos problemas individualmente, estarei utilizando slices de receitas_np, afim de utilizar arrays menores em cada situação.

## No problema 1, deve-se determinar a quais categorias pertencem os pratos mais calóricos. Decidi inicialmente fazer um slice em receitas_np, para utilizar somente as 3 primeiras colunas. Essas são as informações que considerei relevantes para a resolução do problema. Além disso, verifiquei os dados presentes nesse subset, para verificar anomalias, como outliers.

In [4]:
data_prob1 = receitas_np[:, :3]
print(f'O valor calórico mínimo é: {np.min(data_prob1[:, 2])}')
print(f'O valor calórico médio é: {np.mean(data_prob1[:, 2])}')
print(f'O valor calórico máximo é: {np.max(data_prob1[:, 2])}')

O valor calórico mínimo é: 0.0
O valor calórico médio é: 6338.838261197786
O valor calórico máximo é: 30111218.0


## Para o problema 1, o valor calórico mínimo pode ter um pequeno impacto no resultado, já o valor calórico máximo tem grande impacto. O valor médio é um bom indicativo de que utilizar os dados desta forma, não é a melhor abordagem, pois as refeições mais calóricas que vemos comumente tem por volta de 2000 calorias. É esperado que o valor médio esteja abaixo disso. Para mitigar este problema, decidi limitar os dados por percentis.

In [5]:
data_prob1_toppercentile = np.percentile(data_prob1[:, 2], 95, 0)
rows_invalid_top = np.where(data_prob1[:, 2] > data_prob1_toppercentile)
data_prob1 = np.delete(data_prob1, rows_invalid_top, 0)
data_prob1_lowpercentile = np.percentile(data_prob1[:, 2], 5, 0)
rows_invalid_low = np.where(data_prob1[:, 2] < data_prob1_lowpercentile)
data_prob1 = np.delete(data_prob1, rows_invalid_low, 0)

In [6]:
print(f'O valor calórico mínimo é: {np.min(data_prob1[:, 2])}')
print(f'O valor calórico médio é: {np.mean(data_prob1[:, 2])}')
print(f'O valor calórico máximo é: {np.max(data_prob1[:, 2])}')

O valor calórico mínimo é: 61.0
O valor calórico médio é: 410.1363858108579
O valor calórico máximo é: 1320.0


In [7]:
categories_unique = []
for i in range(len(data_prob1)):
    categories_unique.append(data_prob1[i, 1])
categories_unique = [item for sublist in categories_unique for item in sublist]
categories_unique = set(categories_unique)
print(len(categories_unique))

658


In [8]:
dF = pd.DataFrame(data_prob1).drop_duplicates(subset=0)
# dF.sort_index(axis=1, ascending=False)
dF.sort_values(by=2)

,0,1,2
8052,Cold Sesame Spinach,"[Side, Steam, Vegetarian, Low/No Sugar, Vinega...",61
2659,Pernod-Marinated New Zealand Mussels with Cher...,"[Garlic, Herb, Shellfish, Appetizer, Steam, Co...",61
1788,Parmesan-Pepper Biscuits,"[Cheese, Pepper, Bake, Picnic, Parmesan, Famil...",61
3980,Twice-Baked Almond Cookies,"[Cookies, Nut, Dessert, Bake, Almond, Spring, ...",61
7361,Stir-Fried Bok Choy,"[Garlic, Ginger, Leafy Green, Vegetable, Side,...",61
...,...,...,...
3209,Fennel-Dusted Chicken with Brown Butter and Ca...,"[Chicken, Quick & Easy, Dinner, Pan-Fry, Caper...",1315
4703,Herbed Yellow Pepper Scrambled Eggs with Chive...,"[Bread, Egg, Pepper, Breakfast, Brunch, Quick ...",1316
3984,Sticky Spicy Ribs,"[Garlic, Ginger, Onion, Pork, Bake, Marinate, ...",1318
8663,Roast Lamb with Lamb Sausage Crust and Fresh G...,"[Olive, Roast, Christmas, Hanukkah, Lamb Chop,...",1319


In [9]:
categories_weight = dict.fromkeys(categories_unique, 0)
for categ in categories_unique:
    for i in range(len(data_prob1)):
        if categ in data_prob1[i, 1]:
            weight = data_prob1[i, 2]
            categories_weight[categ] += weight            

In [10]:
from collections import Counter

dict(Counter(categories_weight).most_common(20))

{'Bon Appétit': 2964031.0,
 'Peanut Free': 2521113.0,
 'Soy Free': 2430168.0,
 'Tree Nut Free': 2063426.0,
 'Gourmet': 1948097.0,
 'Vegetarian': 1785233.0,
 'Kosher': 1706507.0,
 'Pescatarian': 1636331.0,
 'Bake': 1483104.0,
 'Quick & Easy': 1459430.0,
 'Wheat/Gluten-Free': 1278584.0,
 'Summer': 1201845.0,
 'Dessert': 1107353.0,
 'Winter': 1050210.0,
 'Fall': 1017512.0,
 'Dinner': 952174.0,
 'No Sugar Added': 833303.0,
 'Dairy Free': 814325.0,
 'Sauté': 739640.0,
 'Side': 719835.0}

## Assim, estes são as categorias com as maiores pontuações em relação a calorias dentre as receitas. Agora, para o problema 2, deveremos dizer os 10 ingredientes mais utilizados nas receitas mais calóricas.

In [11]:
data_prob2 = receitas_np[:, :4]

data_prob2_toppercentile = np.percentile(data_prob2[:, 2], 95, 0)
rows_invalid_top = np.where(data_prob2[:, 2] > data_prob2_toppercentile)
data_prob2 = np.delete(data_prob2, rows_invalid_top, 0)
data_prob2_lowpercentile = np.percentile(data_prob2[:, 2], 5, 0)
rows_invalid_low = np.where(data_prob2[:, 2] < data_prob2_lowpercentile)
data_prob2 = np.delete(data_prob2, rows_invalid_low, 0)

# rows_none, _ = np.where(data_prob2 == None)
# data_prob2 = np.delete(data_prob2, rows_none, 0)

# data_prob2_95percentile = np.percentile(data_prob2[:, 2], 95, 0)
# rows_invalid_95 = np.where(data_prob2[:, 2] > data_prob2_95percentile)
# data_prob2 = np.delete(data_prob2, rows_invalid_95, 0)
# data_prob2_5percentile = np.percentile(data_prob2[:, 2], 5, 0)
# rows_invalid_5 = np.where(data_prob2[:, 2] < data_prob2_5percentile)
# data_prob2 = np.delete(data_prob2, rows_invalid_5, 0)

## Agora, vamos visualizar os dados:

In [12]:
df = pd.DataFrame(data_prob2).drop_duplicates(subset=0)
data_prob2 = df.to_numpy()
display(df.rename(columns={
    0: 'title',
    1: 'categories',
    2: 'calories',
    3: 'ingredients'
}).sort_values('calories'))

,title,categories,calories,ingredients
8052,Cold Sesame Spinach,"[Side, Steam, Vegetarian, Low/No Sugar, Vinega...",61,[a 10-ounce bag fresh spinach or 2 small bunch...
2659,Pernod-Marinated New Zealand Mussels with Cher...,"[Garlic, Herb, Shellfish, Appetizer, Steam, Co...",61,"[2 garlic cloves, 1/2 cup plus 2 tablespoons o..."
1788,Parmesan-Pepper Biscuits,"[Cheese, Pepper, Bake, Picnic, Parmesan, Famil...",61,[1 cup grated Parmigiano-Reggiano or Pecorino ...
3980,Twice-Baked Almond Cookies,"[Cookies, Nut, Dessert, Bake, Almond, Spring, ...",61,"[2 1/2 cups all purpose flour, Pinch of salt, ..."
7361,Stir-Fried Bok Choy,"[Garlic, Ginger, Leafy Green, Vegetable, Side,...",61,"[1 pound young bok choy, 2 tablespoons chicken..."
...,...,...,...,...
3209,Fennel-Dusted Chicken with Brown Butter and Ca...,"[Chicken, Quick & Easy, Dinner, Pan-Fry, Caper...",1315,"[1 tablespoon fennel seeds, finely ground, 4 c..."
4703,Herbed Yellow Pepper Scrambled Eggs with Chive...,"[Bread, Egg, Pepper, Breakfast, Brunch, Quick ...",1316,[2 small brioches (about 4 by 3 inches or four...
3984,Sticky Spicy Ribs,"[Garlic, Ginger, Onion, Pork, Bake, Marinate, ...",1318,"[2 tablespoons packed dark brown sugar, 1 1/2 ..."
8663,Roast Lamb with Lamb Sausage Crust and Fresh G...,"[Olive, Roast, Christmas, Hanukkah, Lamb Chop,...",1319,"[2 cups halved seedless red grapes, 2 cups taw..."


In [31]:
ingredients_unique = []
for i in range(len(data_prob2)):
    for j in range(len(data_prob2[i, 3])):
        word = data_prob2[i, 3][j].split()
        for k in range(len(word)):
            try:
                float(word[k])
                continue
            except ValueError:
                ingredients_unique.append(word)
ingredients_unique = [item for sublist in ingredients_unique for item in sublist]
print(len(ingredients_unique))
print(ingredients_unique[:100])

4980370
['4', 'cups', 'low-sodium', 'vegetable', 'or', 'chicken', 'stock', '4', 'cups', 'low-sodium', 'vegetable', 'or', 'chicken', 'stock', '4', 'cups', 'low-sodium', 'vegetable', 'or', 'chicken', 'stock', '4', 'cups', 'low-sodium', 'vegetable', 'or', 'chicken', 'stock', '4', 'cups', 'low-sodium', 'vegetable', 'or', 'chicken', 'stock', '4', 'cups', 'low-sodium', 'vegetable', 'or', 'chicken', 'stock', '1', 'cup', 'dried', 'brown', 'lentils', '1', 'cup', 'dried', 'brown', 'lentils', '1', 'cup', 'dried', 'brown', 'lentils', '1', 'cup', 'dried', 'brown', 'lentils', '1/2', 'cup', 'dried', 'French', 'green', 'lentils', '1/2', 'cup', 'dried', 'French', 'green', 'lentils', '1/2', 'cup', 'dried', 'French', 'green', 'lentils', '1/2', 'cup', 'dried', 'French', 'green', 'lentils', '1/2', 'cup', 'dried', 'French', 'green', 'lentils', '1/2', 'cup', 'dried', 'French', 'green', 'lentils', '2', 'stalks']


In [30]:
a='1'
float(a)

1.0

In [27]:
dict(Counter(ingredients_unique).most_common(20))

{'1': 228539,
 'cup': 148896,
 '1/2': 140957,
 '2': 128234,
 'or': 80405,
 'and': 75736,
 'fresh': 74675,
 'chopped': 74585,
 'tablespoons': 74335,
 '1/4': 67660,
 'teaspoon': 65862,
 'into': 57046,
 'cups': 55374,
 'cut': 55058,
 '3': 52322,
 '(about': 40096,
 '4': 38721,
 'large': 37731,
 'to': 35880,
 '3/4': 33682}

In [ ]:
# ingredients_weight = dict.fromkeys(ingredients_unique, 0)
# for ing in ingredients_unique:
#     for i in range(len(data_prob2)):
#         if ing in data_prob2[i, 3]:
#             weight = data_prob2[i, 2]
#             ingredients_weight[ing] += weight 

In [ ]:
# print(dict(Counter(ingredients_weight).most_common(20)))